In [3]:
using BeliefPropagation
using BeliefPropagation.Models
using BeliefPropagation.FactorGraphs

[ Info: Precompiling BeliefPropagation [f47dce41-c8ad-4b69-b8b4-f2a36c7ebdf8]


In [4]:
import BeliefPropagation: ConvergenceChecker
struct BeliefSignConvergence{F<:Real,T<:Integer} <: ConvergenceChecker
    b       :: Vector{F}    
    ntarget :: T
    n       :: Vector{T}
end
function belief_sign_convergence(bp, ntarget)
    BeliefSignConvergence(copy(bp.b), ntarget, [0])
end

function (convergence_check::BeliefSignConvergence)(bp, errv, errf, errb)
    bnew = bp.b
    bold = convergence_check.b
    unchanged = !any(sign(bnewi) != sign(boldi) for (bnewi, boldi) in zip(bnew, bold))
    if unchanged 
        convergence_check.n[1] += 1
    else
        convergence_check.n[1] = 0
    end 
    convergence_check.b .= bnew
    return convergence_check.n[1] == convergence_check.ntarget 
end

In [5]:
function nunsat(bp::BP)
    @assert !any(isequal(0), bp.b)
    sum(Int((1 - prod(sign, bp.b[i] for i in neighbors(bp.g, factor(a)))) / 2)
        for a in factors(bp.g))
end

function solve_xorsat(n::Integer, γ::Real, k::Integer=3; bp_args...)
    m = round(Int, n*γ)
    g = rand_regular_factor_graph(n, m, k) 
    ψ = fill(IsingCoupling(1), m)
    ϕ = IsingField.(1e-8 * randn(n))
    bp = fast_ising_bp(g, ψ, ϕ)
    iters = iterate_ms!(bp; check_convergence=belief_sign_convergence(bp, 5), bp_args...)
    return nunsat(bp)
end

function sweep(ns, γs, k::Integer=3; nsamples=fill(10^2, length(ns)), bp_args...)
    map(ns, nsamples) do n, nsam
        println("Size $n")
        map(γs) do γ
            println("\tDensity $γ")
            map(1:nsam) do _
                E = solve_xorsat(n, γ, k; bp_args...)
                E / n
            end
        end
    end
end

sweep (generic function with 2 methods)

In [6]:
ns = Int.([1e2, 1e3, 1e4])
γs = LinRange(0.7,0.9,10)
nsamples = [500, 50, 10] 
es = sweep(ns, γs; nsamples, maxiter=10^3, tol=0.0, rein=1e-3);

Size 100
	Density 0.7


In [ ]:
using Statistics, Plots, ColorSchemes

cg = cgrad(:copper, length(ns), categorical=true)
pl = plot(xlabel="γ", ylabel="E")
for (i, en) in pairs(es)
    n = ns[i]
    plot!(pl, γs, mean.(en), yerr=std.(en) ./ sqrt.(length.(en)), 
        label="n=$n", c=cg[i], lw=1.5)
end
pl

UndefVarError: UndefVarError: `es` not defined